In [ ]:
import pandas as pd
from statsmodels.multivariate.manova import MANOVA


In [ ]:
df=pd.read_csv('/content/archive (23).zip')
df

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,Iris-virginica
146,147,6.3,2.5,5.0,1.9,Iris-virginica
147,148,6.5,3.0,5.2,2.0,Iris-virginica
148,149,6.2,3.4,5.4,2.3,Iris-virginica


Manova

In [ ]:
maov = MANOVA.from_formula(  'Species~SepalLengthCm + SepalWidthCm + PetalLengthCm + PetalWidthCm	', data=df)
print(maov.mv_test())

                               Multivariate linear model
                                                                                        
----------------------------------------------------------------------------------------
       Intercept               Value        Num DF  Den DF         F Value        Pr > F
----------------------------------------------------------------------------------------
          Wilks' lambda             -0.0000 3.0000 143.0000 -155109524738482.3750 1.0000
         Pillai's trace              1.0000 3.0000 143.0000 -155109524738482.3750 1.0000
 Hotelling-Lawley trace -3254045973534.5952 3.0000 143.0000 -155109524738482.3750 1.0000
    Roy's greatest root -3254045973534.5952 3.0000 143.0000 -155109524738482.3750 1.0000
----------------------------------------------------------------------------------------
                                                                                        
-----------------------------------------------------

 Without using MANOVA command

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import f

In [ ]:
# Load the Iris dataset
df = pd.read_csv('/content/archive (23).zip')

In [ ]:
# Separate data by species
groups = df.groupby('Species')

In [ ]:
# Calculate means
means = groups.mean()
overall_mean = df.iloc[:, 1:-1].mean()
overall_mean

,0
SepalLengthCm,5.843333
SepalWidthCm,3.054000
PetalLengthCm,3.758667
PetalWidthCm,1.198667


In [ ]:
# Calculate between-group and within-group scatter matrices
S_b = np.zeros((4, 4))
S_w = np.zeros((4, 4))

for name, group in groups:
    n = len(group)
    # Select only the feature columns for the mean difference calculation
    mean_diff = (means.loc[name, ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']] - overall_mean).values.reshape(4, 1)
    S_b += n * (mean_diff @ mean_diff.T)
    # Calculate covariance, handle potential NaNs by filling with 0
    cov_matrix = group.iloc[:, 1:-1].cov().values
    cov_matrix = np.nan_to_num(cov_matrix)  # Replace NaNs and infs with 0
    S_w += cov_matrix * (n - 1)

In [ ]:
# Calculate test statistic
S_w_inv = np.linalg.inv(S_w)
eigvals = np.linalg.eigvals(S_w_inv @ S_b)
wilks_lambda = np.prod(1 / (1 + eigvals))



In [ ]:
# Degrees of freedom
df1 = 4 * (len(groups) - 1)
df2 = 4 * (len(df) - len(groups))

In [ ]:
# Convert Wilks' Lambda to F-statistic
f_stat = ((1 - wilks_lambda) / wilks_lambda) * (df2 / df1)
p_value = 1 - f.cdf(f_stat, df1, df2)

In [ ]:
print(f"Wilks' Lambda: {wilks_lambda}")
print(f"F-statistic: {f_stat}")
print(f"P-value: {p_value}")

Wilks' Lambda: 0.02352545352134156
F-statistic: 3050.7755823314133
P-value: 1.1102230246251565e-16


In [ ]:
# Check significance at 1% and 10% levels
alpha_1 = 0.01
alpha_10 = 0.10
print(f"Significant at 1% level: {p_value < alpha_1}")
print(f"Significant at 10% level: {p_value < alpha_10}")

Significant at 1% level: True
Significant at 10% level: True
